In [7]:
# -*- coding: utf-8 -*-
!pip install --quiet gspread gspread-dataframe oauth2client pandas requests

import pandas as pd
import requests
import gspread
import re
from oauth2client.service_account import ServiceAccountCredentials

# === KONSTANTA ===
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/12taGrqZYfb9F91eIo9q2oVKsnpxAIqFbVFOya1LSxyg/edit"
OLLAMA_URL = "https://fragrances-marilyn-andy-eternal.trycloudflare.com"
OLLAMA_MODEL = "llama3.2:3b"

# === AUTENTIKASI GSPREAD ===
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/credentials.json", scope)
client = gspread.authorize(creds)

# === AKSES SHEETS ===
sheet = client.open_by_url(SPREADSHEET_URL)
wawancara_ws = sheet.worksheet("Hasil Wawancara")
rubrik_ws = sheet.worksheet("Penilaian Rubrik")

# === AMBIL DATA ===
df_wawancara = pd.DataFrame(wawancara_ws.get_all_values())
df_wawancara.columns = df_wawancara.iloc[0]
df_wawancara = df_wawancara[1:]

rubrik_data = rubrik_ws.get_all_values()
rubrik_header = rubrik_data[0]
rubrik_rows = rubrik_data[1:]

# === GABUNG JAWABAN DESKRIPTIF UMKM ===
def gabung_deskripsi(row):
    return ' '.join([str(x).strip() for x in row if str(x).strip()])

# === ANALISIS DENGAN OLLAMA ===
def analyze_skor(kriteria, rubrik, deskripsi):
    prompt = f"""
Anda bertindak sebagai evaluator digitalisasi UMKM berbasis rubrik penilaian. Tugas Anda adalah membaca deskripsi UMKM di bawah ini dan menilai seberapa baik UMKM tersebut memenuhi kriteria yang ditentukan.

=== DESKRIPSI UMKM ===
{deskripsi}

=== KRITERIA PENILAIAN ===
{kriteria}

=== RUBRIK PENILAIAN ===
{rubrik}

=== PETUNJUK PENILAIAN ===
Berdasarkan deskripsi UMKM dan rubrik yang diberikan, tentukan skor digitalisasi UMKM sesuai pedoman berikut:
- 0: Tidak ada bukti digitalisasi
- 1: Terdapat upaya awal digitalisasi namun masih terbatas
- 2: Implementasi digitalisasi sebagian, sudah mulai terintegrasi
- 3: Digitalisasi menyeluruh, sudah menjadi bagian integral dari kegiatan usaha

Berikan **hanya satu angka** dari 0 hingga 3 sesuai penilaian Anda. **Jangan tambahkan penjelasan, alasan, atau kata lain apa pun** dalam jawaban.
"""
    try:
        res = requests.post(
            f"{OLLAMA_URL}/api/generate",
            json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False},
            timeout=240
        )
        output = res.text.strip()
        match = re.search(r'\b[0-3]\b', output)
        return match.group(0) if match else "0"
    except Exception as e:
        print(f"❌ Error saat analisis kriteria '{kriteria}': {e} → default = 0")
        return "0"

# === PROSES SETIAP PT YANG VALID ===
jumlah_umkm = 0

for idx, row in df_wawancara.iterrows():
    nama_industri = row['Nama Industri']
    if not nama_industri.strip():
        print(f"⏹️ Baris {idx+2} kosong → proses berhenti.")
        break

    kolom_output = chr(ord('C') + jumlah_umkm)
    deskripsi = gabung_deskripsi(row.iloc[5:23])  # Kolom F–W

    print(f"\n🔍 Memproses '{nama_industri}' → Kolom {kolom_output}")

    for i, rubrik_row in enumerate(rubrik_rows):
        kriteria = rubrik_row[0]
        rubrik = rubrik_row[1]
        skor = analyze_skor(kriteria, rubrik, deskripsi)
        rubrik_ws.update_acell(f"{kolom_output}{i + 2}", skor)

    jumlah_umkm += 1

print("\n✅ Semua skor berhasil ditulis ke Sheet 'Penilaian Rubrik'")



🔍 Memproses 'PT CIFA Indonesia' → Kolom C

🔍 Memproses 'PT Mual Tio' → Kolom D

🔍 Memproses 'CV AZK Mekar Mulya' → Kolom E

✅ Semua skor berhasil ditulis ke Sheet 'Penilaian Rubrik'
